In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df

,tweet
0,@McDonalds the facility that you use was very ...
1,@McDonalds thank you very much for the nice ha...
2,@McDonalds <a2323++++> facility is too BAD and...
3,@McDonalds give u a very BAD BAD rating on fac...
4,@McDonalds your Facility was so bad
5,@McDonalds your hamburger is very delicious
6,@McDonalds Thank you for the NICE Hamburger…...


In [ ]:
list_data = []

for i in df.tweet:
  list_data.append(i)

In [ ]:
list_data

['@McDonalds the facility that you use was very bad and make me so angry',
 '@McDonalds thank you very much for the nice hamburger',
 '@McDonalds <a2323++++> facility is too BAD and BAD',
 '@McDonalds give u a very BAD BAD rating on facility',
 '@McDonalds your Facility was so bad',
 '@McDonalds your hamburger is very delicious',
 '@McDonalds Thank you for the NICE Hamburger…...']

In [ ]:
!pip install scikit-learn

In [ ]:
import string
import re

list_data_bersih1 = []

pat1 = r'@[A-Za-z0-9_]+' #deleting @mention
pat2 = r'https?://[A-Za-z0-9./]+' #delete url
pat3 = r'#[A-Za-z0-9]+' #delete hashtag
pat4 = r'\[A-Za-z0-9]+'#delete alphabeth+number
pat5 = r'\<[A-Za-z0-9+_>]+' #deleting error mp
pat6 = r'[0-9]+' #delete full of number with more than 1
pattern = re.compile('\W')  #deleting white space

combined_pat = r'|'.join((pat1, pat2, pat3,pat4, pat5, pat6)) #masking creating

for j in list_data:
  dataBersih1 = re.sub(combined_pat,'',str(j)) #filtering process
  list_data_bersih1.append(dataBersih1) #store cleaning data

In [ ]:
list_data_bersih1

[' the facility that you use was very bad and make me so angry',
 ' thank you very much for the nice hamburger',
 '  facility is too BAD and BAD',
 ' give u a very BAD BAD rating on facility',
 ' your Facility was so bad',
 ' your hamburger is very delicious',
 ' Thank you for the NICE Hamburger…...']

In [ ]:
list_data_bersih = []

for k in list_data_bersih1: 
  k = k.lower()  
  dataBersih2 = re.sub(pattern,' ',str(k)) #menghapus karakter non alphanumeric
  dataBersih3 = re.sub(r'\b\w{1,1}\b', '', dataBersih2) #mengapus karakter 1 digit
  list_data_bersih.append(dataBersih3) #menyimpan dalam list

list_data_bersih

[' the facility that you use was very bad and make me so angry',
 ' thank you very much for the nice hamburger',
 '  facility is too bad and bad',
 ' give   very bad bad rating on facility',
 ' your facility was so bad',
 ' your hamburger is very delicious',
 ' thank you for the nice hamburger    ']

In [ ]:
#vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=0.005, max_df=0.99, use_idf=True, ngram_range=(1,3), smooth_idf=False, norm='l2')

In [ ]:
X = vect.fit_transform(list_data_bersih)

In [ ]:
Z = pd.DataFrame(X.toarray(), columns=vect.get_feature_names(), index=['tweet %d' %i for i in range (1, X.shape[0]+1)])
Z

,and,and bad,and make,and make me,angry,bad,bad and,bad and bad,bad and make,bad bad,bad bad rating,bad rating,bad rating on,delicious,facility,facility is,facility is too,facility that,facility that you,facility was,facility was so,for,for the,for the nice,give,give very,give very bad,hamburger,hamburger is,hamburger is very,is,is too,is too bad,is very,is very delicious,make,make me,make me so,me,me so,...,thank you for,thank you very,that,that you,that you use,the,the facility,the facility that,the nice,the nice hamburger,too,too bad,too bad and,use,use was,use was very,very,very bad,very bad and,very bad bad,very delicious,very much,very much for,was,was so,was so bad,was very,was very bad,you,you for,you for the,you use,you use was,you very,you very much,your,your facility,your facility was,your hamburger,your hamburger is
tweet 1,0.138351,0.00000,0.18092,0.18092,0.18092,0.095782,0.138351,0.00000,0.18092,0.000000,0.000000,0.000000,0.000000,0.000000,0.095782,0.00000,0.00000,0.18092,0.18092,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.18092,0.18092,0.18092,0.18092,0.18092,...,0.000000,0.000000,0.18092,0.18092,0.18092,0.113450,0.18092,0.18092,0.000000,0.000000,0.00000,0.00000,0.00000,0.18092,0.18092,0.18092,0.095782,0.138351,0.18092,0.000000,0.000000,0.000000,0.000000,0.138351,0.00000,0.00000,0.18092,0.18092,0.113450,0.000000,0.000000,0.18092,0.18092,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
tweet 2,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.199296,0.199296,0.199296,0.000000,0.000000,0.000000,0.163426,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.260617,0.00000,0.00000,0.00000,0.163426,0.00000,0.00000,0.199296,0.199296,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.137975,0.000000,0.00000,0.000000,0.000000,0.260617,0.260617,0.000000,0.00000,0.00000,0.00000,0.00000,0.163426,0.000000,0.000000,0.00000,0.00000,0.260617,0.260617,0.000000,0.00000,0.00000,0.000000,0.000000
tweet 3,0.219334,0.28682,0.00000,0.00000,0.00000,0.303695,0.219334,0.28682,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.151847,0.28682,0.28682,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.219334,0.28682,0.28682,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.28682,0.28682,0.28682,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
tweet 4,0.000000,0.00000,0.00000,0.00000,0.00000,0.270993,0.000000,0.00000,0.00000,0.255936,0.255936,0.255936,0.255936,0.000000,0.135497,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.255936,0.255936,0.255936,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.135497,0.195716,0.00000,0.255936,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000
tweet 5,0.000000,0.00000,0.00000,0.00000,0.00000,0.173464,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.173464,0.00000,0.00000,0.00000,0.00000,0.32765,0.32765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.00000,0

In [ ]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=6, algorithm='randomized')
lsa_matrix = lsa.fit_transform(X)

In [ ]:
import numpy as np
Y = pd.DataFrame(np.array(lsa_matrix.tolist()), columns=['PC %d' %i for i in range(1,7)], index=['tweet %d' %i for i in range (1, X.shape[0]+1)])
Y

,PC 1,PC 2,PC 3,PC 4,PC 5,PC 6
tweet 1,0.190795,0.522172,-0.320986,-0.402427,0.317022,0.570479
tweet 2,0.863433,-0.147772,-0.037959,0.022368,-0.029652,-0.052528
tweet 3,0.076887,0.584450,-0.045046,0.322647,0.557497,-0.485339
tweet 4,0.085014,0.510598,-0.267379,0.571403,-0.543106,0.197046
tweet 5,0.074123,0.541002,0.179085,-0.566796,-0.437855,-0.395934
tweet 6,0.170897,0.248575,0.882842,0.159067,0.066065,0.315831
tweet 7,0.858997,-0.166430,-0.051150,-0.001266,-0.012120,-0.078641


In [ ]:
from sklearn.cluster import KMeans
model = KMeans (n_clusters=2, max_iter=100, random_state=100)
model.fit(lsa_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=100, tol=0.0001, verbose=0)

In [ ]:
clust = model.labels_.tolist()
clust

[0, 1, 0, 0, 0, 1, 1]

In [ ]:
cek_data = {'Tweet' : list_data, 'Cluster Code' : clust}
cek_data

{'Cluster Code': [0, 1, 0, 0, 0, 1, 1],
 'Tweet': ['@McDonalds the facility that you use was very bad and make me so angry',
  '@McDonalds thank you very much for the nice hamburger',
  '@McDonalds <a2323++++> facility is too BAD and BAD',
  '@McDonalds give u a very BAD BAD rating on facility',
  '@McDonalds your Facility was so bad',
  '@McDonalds your hamburger is very delicious',
  '@McDonalds Thank you for the NICE Hamburger…...']}

In [ ]:
dataframe = pd.DataFrame(cek_data)
dataframe

,Tweet,Cluster Code
0,@McDonalds the facility that you use was very ...,0
1,@McDonalds thank you very much for the nice ha...,1
2,@McDonalds <a2323++++> facility is too BAD and...,0
3,@McDonalds give u a very BAD BAD rating on fac...,0
4,@McDonalds your Facility was so bad,0
5,@McDonalds your hamburger is very delicious,1
6,@McDonalds Thank you for the NICE Hamburger…...,1


In [ ]:
centers = lsa.inverse_transform(model.cluster_centers_).argsort()[:, ::-1]

terms = vect.get_feature_names()
kata = []
for i in range(0, 2):
  l= []
  l.append(", ".join([terms[c] for c in centers[i, :3]]))
  kata.append(l)

In [ ]:
kata

[['bad, facility, so'], ['hamburger, for the nice, for']]

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
a = 'This was a VERY BAD movie.'
sid.polarity_scores(a)

{'compound': -0.7398, 'neg': 0.568, 'neu': 0.432, 'pos': 0.0}